

```
Computer vision, OpenEdu Course
```



In [ ]:
import cv2
import numpy as np

# Пути к файлам
cfg_file = '/content/yolov3.cfg'
weights_file = '/content/yolov3.weights'
names_file = '/content/coco.names'
video_file = '/content/var_MOT17-04-SDP_4.mp4'

# Загрузка имен классов
with open(names_file, 'r') as f:
    classes = f.read().strip().split('\n')

# Загрузка сети YOLO
net = cv2.dnn.readNetFromDarknet(cfg_file, weights_file)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Загрузка видео
cap = cv2.VideoCapture(video_file)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Параметры NMS
score_threshold = 0.5
nms_threshold = 0.3

# Инициализация переменных для анализа
max_objects = 0
max_classes = 0
frame_number = 89
frame_with_max_confidence = None
max_confidence = 0
best_box = None
best_class_id = None

for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # Создание блоба из кадра и выполнение прямого прохода
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    # Пост-обработка выходов сети
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > score_threshold:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Применение NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold, nms_threshold)

    detected_classes = set()
    detected_objects = 0
    for ind in indices:
        detected_objects += 1
        detected_classes.add(class_ids[ind])
        if i == frame_number and confidences[ind] > max_confidence:
            max_confidence = confidences[ind]
            best_box = boxes[ind]
            best_class_id = class_ids[ind]

    if detected_objects > max_objects:
        max_objects = detected_objects

    if len(detected_classes) > max_classes:
        max_classes = len(detected_classes)

cap.release()

# Вывод результатов
print("Количество кадров в видео:", frame_count)
print("Максимальное количество распознанных объектов в кадре:", max_objects)
print("Максимальное количество классов в кадре:", max_classes)
print("Кадр 89:")
print("Вероятность отнесения объекта к классу:", max_confidence)
print("Метка класса:", classes[best_class_id])
print("Координата x левого верхнего угла:", best_box[0])
print("Координата y левого верхнего угла:", best_box[1])
print("Ширина прямоугольника:", best_box[2])
print("Высота прямоугольника:", best_box[3])


Количество кадров в видео: 100
Максимальное количество распознанных объектов в кадре: 27
Максимальное количество классов в кадре: 4
Кадр 89:
Вероятность отнесения объекта к классу: 0.9918854832649231
Метка класса: person
Координата x левого верхнего угла: 401
Координата y левого верхнего угла: 641
Ширина прямоугольника: 97
Высота прямоугольника: 249
